In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.graph_objs as go
import pandas as pd
from datetime import datetime

import yfinance as yf

# Read data
df = pd.read_csv('../DATA/chilean_stocks.csv', index_col='Date')
df = df.groupby(by=['Ticker', 'Name']).sum().reset_index()[['Ticker','Name']].set_index('Ticker')

available_tickers = []
for tic in df.index:
    available_tickers.append({'label':df.loc[tic]['Name'], 'value':tic})

# Release memory
del df

app = dash.Dash(
    __name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}]
)

available_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Pct Change - Close', 'Cummulative Returns']

app.layout = html.Div([
    html.H1('Bolsa de Santiago - Principales Acciones - Dashboard',
            style={'textAlign': 'center'},
           ),
    html.Div([html.H3(html.Label('Seleccionar la Acción')),
            dcc.Dropdown(
                id='ticker-selected',
                options=available_tickers,
                value='LTM.SN',
                multi=True,
                placeholder="Acción",
                clearable=False,
            )
    ],style={'width': '48%', 'display': 'inline-block'},),
    html.Div([html.H3(html.Label('Elija Información a visualizar')),
            dcc.Dropdown(
                id='y-selected',
                options=[{'label': i, 'value': i} for i in available_columns],
                value='Close',
                placeholder="Información a visualizar",
                clearable=False,
            )
    ],style={'width': '48%', 'display': 'inline-block'},),
    html.Div([
        html.H3('Seleccione fecha de inicio y final:'),
        dcc.DatePickerRange(
            id='my_date_picker',
            min_date_allowed=datetime(2018, 1, 1),
            max_date_allowed=datetime.today(),
            start_date=datetime(2020, 1, 1),
            end_date=datetime.today()
        )
    ], style={'display':'inline-block'}),
    html.Div([
        html.Button(
            id='submit-button',
            n_clicks=0,
            children='Submit',
            style={'fontSize':24, 'marginLeft':'30px'}
        ),
    ], style={'display':'inline-block'}),
    # Graph 
    html.Hr(),
    html.Div([dcc.Graph(id='graph-stock',
                        figure={
                            'data': [
                                {'x': [0,0], 'y': [0,0]}
                            ]
                        }),                         
             ],),
    html.Div([dcc.Graph(id='table-stock',
                        figure={
                            'data': [
                                {'x': [0,0], 'y': [0,0]}
                            ]
                        }),                         
             ],),
    html.Hr(),
])

@app.callback(Output('graph-stock', 'figure'),
              [Input('submit-button', 'n_clicks')],
              [State('ticker-selected', 'value'),
               State('y-selected', 'value'),
               State('my_date_picker', 'start_date'),
               State('my_date_picker', 'end_date'),
              ]
             )


def update_graph(n_clicks, tickers, col, start_date, end_date):
    start = datetime.strptime(start_date[:10], '%Y-%m-%d')
    end = datetime.strptime(end_date[:10], '%Y-%m-%d')
    traces = []
    title = ''
    if type(tickers)==str: tickers = [tickers]
    #tickers.append('BOLSASTGO.SN')
    for accion in tickers:
        df_ticker = yf.Ticker(accion)
        # get company name

        try:
            longname = df_ticker.info['longName']
            industry = df_ticker.info['industry']
            payoutRatio = df_ticker.info['payoutRatio']
        except:
            longname = accion
            industry = 'N/A'
            payoutRatio = 0 
        # get historical market data
        df = df_ticker.history(start=start, end=end, interval='1d')
        df['Ticker'] = accion
        df['Name'] = longname
        df['Industry'] = industry
        df['PayoutRatio'] = payoutRatio
        df['Pct Change - Close'] = df.Close.pct_change()
        df['Cummulative Returns'] = ((1 + df['Pct Change - Close']).cumprod() - 1)    

        data = go.Scatter(x = df.index, y=df[col], mode='lines+markers', 
                          name=accion,
                          text=longname, 
                          hovertext=["x", "text", col],
                          hoverinfo=["y"],
                          xaxis='x2',
                          yaxis='y2',
                          marker=dict(symbol = 'pentagon-dot',
                                      opacity = 0.8,
                                      line   = dict(width=2),
                                     ),
                         )
        traces.append(data)
        title = title + accion+'/'
    layout = go.Layout(title = title+' - '+'<b>'+col+'</b>'+'<br>'+'Desde : '+start_date[:10] \
                       + ' Hasta : '+end_date[:10],
                       title_x=0.5,
                       xaxis = dict(title='Fecha'),
                       yaxis = dict(title=col),
                       hovermode="x unified",
                       hoverlabel=dict(bgcolor="white", 
                                       font_size=10, 
                                       font_family="Rockwell"
                                      ),
                       template='presentation',
                       height=600,
                       autosize=True,  
                      )
    fig = go.Figure(data=traces,layout=layout)
    fig.update_xaxes(rangeslider_visible=True,
                     rangeselector=dict(buttons=list([
                         dict(count=1, label="1m", step="month", stepmode="backward"),
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="YTD", step="year", stepmode="todate"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=3, label="3y", step="year", stepmode="backward"),
                         dict(count=5, label="5y", step="year", stepmode="backward"),
                         dict(step="all")])))
    return fig

@app.callback(Output('table-stock', 'figure'),
              [Input('submit-button', 'n_clicks')],
              [State('ticker-selected', 'value'),
               State('y-selected', 'value'),
               State('my_date_picker', 'start_date'),
               State('my_date_picker', 'end_date'),
              ]
             )

def update_table(n_clicks, tickers, col, start_date, end_date):
    start = datetime.strptime(start_date[:10], '%Y-%m-%d')
    end = datetime.strptime(end_date[:10], '%Y-%m-%d')
    df_table = pd.DataFrame()
    if type(tickers)==str: tickers = [tickers]
    #tickers.append('BOLSASTGO.SN')
    for accion in tickers:
        df_ticker = yf.Ticker(accion)
        # get company name
        try:
            longname = df_ticker.info['longName']
            industry = df_ticker.info['industry']
            payoutRatio = df_ticker.info['payoutRatio']
        except:
            longname = accion
            industry = 'N/A'
            payoutRatio = 0 
        # get historical market data
        df = df_ticker.history(start=start, end=end, interval='1d')
        df['Ticker'] = accion
        df['Name'] = longname
        df['Industry'] = industry
        df['PayoutRatio'] = payoutRatio
        df['Pct Change - Close'] = df.Close.pct_change()
        df['Cummulative Returns'] = ((1 + df['Pct Change - Close']).cumprod() - 1)
        df_table = df_table.append(df)
    
    df_table.reset_index(inplace=True)
    df_table.sort_values(["Name", "Date"], axis=0, ascending=False, inplace=True) 
    df_table['Fecha'] = df_table['Date'].apply(lambda x:datetime.strftime(x,'%d/%m/%Y'))
    df_table = df_table[['Fecha','Name','Industry', 'Open', 'High', 'Low', 'Close', 'Volume', 'Pct Change - Close','Cummulative Returns']]
    table = go.Table(header=dict(values=list(df_table.columns),
                                 font=dict(size=10),
                                 #fill_color='paleturquoise',
                                 align='left'),
                     cells=dict(values=[df_table[k].tolist() for k in df_table.columns],
                                align = "left",
                                font=dict(size=10),
                               ),
                    )
    layout = go.Layout(title = 'Detalle de Información - '+'<b>'+col+'<b>', title_x=0.5,)
    fig = go.Figure(data=table, layout=layout)
    
    return fig
              
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2020 00:37:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 00:37:41] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_4_1m1592228780.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 00:37:41] "GET /_dash-component-suites/dash_renderer/react@16.v1_4_1m1592228780.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 00:37:41] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_4_1m1592228780.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 00:37:41] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_4_1m1592228780.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 00:37:41] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_10_0m1592228781.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 00:37:41] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_0_3m1592228781.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2020 00

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.graph_objs as go
import pandas as pd
from datetime import datetime

import yfinance as yf

In [2]:
df_ticker = yf.Ticker('LTM.SN')

In [3]:
df_ticker.info

{'sector': 'Industrials',
 'fullTimeEmployees': 42589,
 'longBusinessSummary': 'LATAM Airlines Group S.A., together with its subsidiaries, provides passenger and cargo air transportation services in Peru, Argentina, the United States, Europe, Colombia, Brazil, Ecuador, Chile, the Asia Pacific, and rest of Latin America. The company provides passenger transport services to approximately 145 destinations in 26 countries, as well as cargo services to approximately 151 destinations in 29 countries; and operates loyalty programs. As of December 31, 2019, it operated a fleet of 331 aircraft, which include 320 passenger aircraft and 11 cargo aircraft; and subleased 11 aircraft comprising 10 passenger aircraft and 1 cargo aircraft to third parties. The company was formerly known as LAN Airlines S.A. and changed its name to LATAM Airlines Group S.A. in June 2012. LATAM Airlines Group S.A. was founded in 1929 and is headquartered in Santiago, Chile. On May 26, 2020, LATAM Airlines Group S.A., al

In [5]:
df_ticker.institutional_holders

,Holder,Shares,Date Reported,% Out,Value
0,Vanguard International Stock Index-Total Intl ...,3115657,2020-04-29,0.0051,9829897835
1,Vanguard International Stock Index-Emerging Ma...,2723294,2020-04-29,0.0045,8591992570
2,iShares MSCI Chile Capped ETF,2645954,2020-02-28,0.0044,14436325024
3,iShares Core MSCI Emerging Markets ETF,2040332,2020-02-28,0.0034,11132051392
4,DFA Investment Dimensions-DFA Emerging Mkts Value,1238786,2020-04-29,0.0020,3908369830
5,iShares MSCI Emerging Markets ETF,1022617,2020-02-28,0.0017,5579398352
6,Schwab Strategic Tr-Schwab Emerging Markets Eq...,384858,2020-02-28,0.0006,2099785248
7,Schwab Strategic Tr-Schwab Fundamental Emg Mkt...,379865,2020-02-28,0.0006,2072543440
8,Vanguard Intl Equity Index Fds-FTSE All World ...,374965,2020-04-29,0.0006,1183014575
9,Cambria ETF Tr-Cambria Global Value ETF,276356,2020-04-29,0.0005,871903180


In [7]:
df_ticker.recommendations

In [8]:
df_ticker.actions

,Dividends,Stock Splits
Date,,
2010-08-16,0.219800,0.0
2011-01-10,184.122000,0.0
2011-05-16,0.030610,0.0
2011-09-12,79.060700,0.0
2012-01-09,128.039000,0.0
2012-05-14,26.605500,0.0
2013-05-13,3.245780,0.0
2017-05-15,22.996900,0.0
2018-05-10,0.076831,0.0


In [12]:
df_ticker.quarterly_financials

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [10]:
df_ticker.major_holders

,0,1
0,65.81%,% of Shares Held by All Insider
1,32.97%,% of Shares Held by Institutions
2,96.43%,% of Float Held by Institutions
3,122,Number of Institutions Holding Shares


In [13]:
df_ticker.balance_sheet

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [14]:
df_ticker.calendar

,0,1
Earnings Date,2020-08-11 00:00:00,2020-08-17 00:00:00
Earnings Average,None,None
Earnings Low,None,None
Earnings High,None,None
Revenue Average,1802100000,1802100000
Revenue Low,1802100000,1802100000
Revenue High,1802100000,1802100000


In [15]:
df_ticker.sustainability

,Value
2020-3,
palmOil,False
controversialWeapons,False
gambling,False
socialScore,18.8
nuclear,False
furLeather,False
alcoholic,False
gmo,False
catholic,False
